In [151]:
import os


In [2]:
pip install jupyter-dash plotly xarray numpy

  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached zipp-3.23.0-py3-none-any.whl.metadata (3.6 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   --------------- ------------------------ 3.9/9.9 MB 21.3 MB/s eta 0:00:01
   ---------------------------------- ----- 8.7/9.9 MB 21.5 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 19.9 MB/s  0:00:00
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ------------------------- -------------- 5.0/7.9 MB 25.2 MB/s eta 0:00:01
   ---------------------------------------- 7.9/7.9 MB 22.3 MB/s  0:00:00
Using cached flask-3.1.2-py3-none-any.whl (103 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Us

# Interactive Dust Forecasting & Visualization Dashboard

## Phase 1: Data Adapter Layer

This notebook provides an interactive scientific visualization interface
for dust source identification, transport analysis, and forecasting.

Phase 1 focuses on transforming final model outputs into a structured,
labeled data format suitable for advanced interactive visualization
using Plotly and JupyterDash.

No model training or preprocessing is performed here.


In [183]:
# Core scientific & visualization libraries
import numpy as np
import xarray as xr

# Plotting (will be used in later phases)
import plotly.graph_objects as go
import plotly.express as px

# Silence warnings for clean output
import warnings
warnings.filterwarnings("ignore")


In [184]:
import os
import numpy as np
import xarray as xr
import plotly.express as px
import plotly.graph_objects as go


In [185]:
import plotly.io as pio
pio.renderers.default = "notebook"


In [186]:
# Load final environmental dust cube
env_cube = np.load("../outputs/environmental_dust_cube.npy")
# Shape: (T, H, W, C)

print("Environmental dust cube shape:", env_cube.shape)

# Sanity check
assert env_cube.ndim == 4, "Expected 4D environmental cube"


Environmental dust cube shape: (14, 721, 1440, 4)


In [187]:
# Extract dimensions
T, H, W, C = env_cube.shape

# Define coordinate labels
time_index = np.arange(T)  # later can be replaced with actual dates
y_coords = np.arange(H)
x_coords = np.arange(W)

# Define variable names (MUST match fusion logic)
variable_names = [
    "NDVI",
    "Wind Speed",
    "Wind Direction",
    "Dust Activity Index"
]

print("Variables:", variable_names)


Variables: ['NDVI', 'Wind Speed', 'Wind Direction', 'Dust Activity Index']


In [188]:
# Convert to xarray Dataset
dust_ds = xr.Dataset(
    {
        var: (("time", "y", "x"), env_cube[:, :, :, i])
        for i, var in enumerate(variable_names)
    },
    coords={
        "time": time_index,
        "y": y_coords,
        "x": x_coords
    }
)

dust_ds


<xarray.Dataset> Size: 233MB
Dimensions:              (time: 14, y: 721, x: 1440)
Coordinates:
  * time                 (time) int32 56B 0 1 2 3 4 5 6 7 8 9 10 11 12 13
  * y                    (y) int32 3kB 0 1 2 3 4 5 6 ... 715 716 717 718 719 720
  * x                    (x) int32 6kB 0 1 2 3 4 5 ... 1435 1436 1437 1438 1439
Data variables:
    NDVI                 (time, y, x) float32 58MB 0.5 0.5 0.5 ... 0.5 0.5 0.5
    Wind Speed           (time, y, x) float32 58MB 6.318 6.318 ... 4.691 4.691
    Wind Direction       (time, y, x) float32 58MB 1.222 1.222 ... -0.3949
    Dust Activity Index  (time, y, x) float32 58MB 3.159 3.159 ... 2.345 2.345

In [189]:
# Basic integrity checks
print("Time steps:", dust_ds.dims["time"])
print("Spatial resolution:", dust_ds.dims["y"], "x", dust_ds.dims["x"])

# Check value ranges
for var in variable_names:
    vmin = float(dust_ds[var].min())
    vmax = float(dust_ds[var].max())
    print(f"{var}: min={vmin:.3f}, max={vmax:.3f}")


Time steps: 14
Spatial resolution: 721 x 1440
NDVI: min=0.000, max=1.000
Wind Speed: min=0.003, max=31.140
Wind Direction: min=-3.142, max=3.142
Dust Activity Index: min=0.000, max=17.343


In [190]:
# Save xarray dataset for reuse
dust_ds.to_netcdf("../outputs/dust_visualization_dataset.nc")

print("Visualization-ready dataset saved")


Visualization-ready dataset saved


In [191]:
dust_ds["Dust Activity Index"].isel(time=0)


<xarray.DataArray 'Dust Activity Index' (y: 721, x: 1440)> Size: 4MB
array([[3.158757 , 3.158757 , 3.158757 , ..., 3.158757 , 3.158757 ,
        3.158757 ],
       [3.4098148, 3.4099536, 3.4107955, ..., 3.4048321, 3.406605 ,
        3.4080129],
       [3.4081883, 3.4100835, 3.4112585, ..., 3.3995564, 3.4023101,
        3.4053178],
       ...,
       [2.6565635, 2.6575196, 2.657116 , ..., 2.6484764, 2.6515045,
        2.6541808],
       [2.5695713, 2.5690126, 2.5681903, ..., 2.5657437, 2.567001 ,
        2.568277 ],
       [2.4328082, 2.4328082, 2.4328082, ..., 2.4328082, 2.4328082,
        2.4328082]], dtype=float32)
Coordinates:
  * y        (y) int32 3kB 0 1 2 3 4 5 6 7 8 ... 713 714 715 716 717 718 719 720
  * x        (x) int32 6kB 0 1 2 3 4 5 6 ... 1433 1434 1435 1436 1437 1438 1439
    time     int32 4B 0

In [192]:
# Phase 2 — Visualization settings

COLOR_SCALES = {
    "NDVI": "YlGn",
    "Wind Speed": "Turbo",
    "Wind Direction": "HSV",
    "Dust Activity Index": "Inferno"
}

DEFAULT_HEIGHT = 550
DEFAULT_WIDTH = 700


In [193]:
# Spatial heatmap visualization

def plot_spatial_map(dataset, variable, time_index, save_html=False):
    import os  # <- BULLETPROOF FIX (local import)

    data = dataset[variable].isel(time=time_index).values

    fig = px.imshow(
        data,
        color_continuous_scale=COLOR_SCALES.get(variable, "Viridis"),
        aspect="auto",
        origin="lower",
        title=f"{variable} — Time Step {time_index}",
    )

    fig.update_layout(
        height=DEFAULT_HEIGHT,
        width=DEFAULT_WIDTH,
        coloraxis_colorbar=dict(title=variable),
        template="plotly_dark"
    )

    if save_html:
        output_dir = "../outputs/visuals"
        os.makedirs(output_dir, exist_ok=True)
        path = f"{output_dir}/{variable.replace(' ', '_')}_t{time_index}.html"
        fig.write_html(path)
        print(f"Saved interactive plot → {path}")

    return fig



In [194]:
import plotly.io as pio
pio.renderers.default = "notebook"


In [195]:
fig = plot_spatial_map(
    dust_ds,
    "Dust Activity Index",
    time_index=0,
    save_html=True
)


Saved interactive plot → ../outputs/visuals/Dust_Activity_Index_t0.html


In [196]:
import plotly.io as pio
pio.renderers.default = "browser"
pio.renderers.render_on_display = False


In [197]:
APP_BG = "#0b0f19"          # Deep dark blue
CARD_BG = "rgba(255,255,255,0.05)"
BORDER_COLOR = "rgba(255,255,255,0.15)"
TEXT_COLOR = "#e6e6e6"
ACCENT = "#00e5ff"          # Cyan accent (AI vibe)


In [198]:
import webbrowser
import threading


In [199]:
# Phase 3 — Dashboard frontend setup

from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px


In [200]:
# Initialize Dash app inside Jupyter
app = JupyterDash(__name__)


In [201]:
# Dashboard layout
app.layout = html.Div(
    style={"backgroundColor": "#111111", "padding": "10px"},
    children=[
        html.H1(
            "Interactive Dust Forecasting & Analysis System",
            style={"textAlign": "center", "color": "white"}
        ),

        html.P(
            "Explore dust sources, transport dynamics, and activity levels "
            "using spatio-temporal environmental data.",
            style={"textAlign": "center", "color": "#CCCCCC"}
        ),

        html.Div(
            style={"display": "flex"},
            children=[
                # LEFT CONTROL PANEL
                html.Div(
                    style={
                        "width": "25%",
                        "padding": "10px",
                        "backgroundColor": "#1e1e1e",
                        "borderRadius": "10px",
                    },
                    children=[
                        html.Label("Select Variable", style={"color": "white"}),
                        dcc.Dropdown(
                            id="variable-dropdown",
                            options=[
                                {"label": v, "value": v}
                                for v in dust_ds.data_vars
                            ],
                            value="Dust Activity Index",
                            clearable=False,
                        ),

                        html.Br(),

                        html.Label("Select Time Step", style={"color": "white"}),
                        dcc.Slider(
                            id="time-slider",
                            min=0,
                            max=len(dust_ds.time) - 1,
                            step=1,
                            value=0,
                            marks=None,
                            tooltip={"placement": "bottom", "always_visible": True},
                        ),
                    ],
                ),

                # MAIN VISUALIZATION
                html.Div(
                    style={"width": "75%", "padding": "10px"},
                    children=[
                        dcc.Graph(id="spatial-map")
                    ],
                ),
            ],
        ),

        html.Hr(style={"borderColor": "#333"}),

        html.Div(
            children=[
                dcc.Graph(id="time-series")
            ]
        )
    ]
)


In [202]:
# Callback to update spatial map
@app.callback(
    Output("spatial-map", "figure"),
    Input("variable-dropdown", "value"),
    Input("time-slider", "value")
)
def update_spatial_map(variable, time_index):

    data = dust_ds[variable].isel(time=time_index).values

    fig = px.imshow(
        data,
        origin="upper",
        aspect="equal",
        color_continuous_scale="Turbo"
    )

    fig.update_layout(
        title=f"{variable} — Time Step {time_index}",
        template="plotly_dark",
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=20, r=20, t=60, b=20),
        coloraxis_colorbar=dict(title=variable),
    )

    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)

    return fig


In [203]:
# Callback to show local time-series when user clicks map
@app.callback(
    Output("time-series", "figure"),
    Input("spatial-map", "clickData"),
    Input("variable-dropdown", "value")
)
def update_time_series(clickData, variable):
    if clickData is None:
        # Default center point
        y = dust_ds.dims["y"] // 2
        x = dust_ds.dims["x"] // 2
    else:
        x = int(clickData["points"][0]["x"])
        y = int(clickData["points"][0]["y"])

    ts = dust_ds[variable][:, y, x].values

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            y=ts,
            mode="lines+markers",
            name=variable,
            line=dict(color="orange", width=2)
        )
    )

    fig.update_layout(
        title=f"{variable} Time-Series at (y={y}, x={x})",
        xaxis_title="Time Index",
        yaxis_title=variable,
        template="plotly_dark",
        height=350
    )

    return fig


In [204]:
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")


In [205]:
if __name__ == "__main__":
    threading.Timer(1, open_browser).start()
    app.run(debug=False)


In [85]:
app.run(debug=False)
